In [17]:
!pip install scorecardpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scorecardpy: filename=scorecardpy-0.1.9.7-py3-none-any.whl size=60629 sha256=28b81021a4f221b457f3024dd8267a44d5385ce11b246b5cad8e7a2903e354b2
  Stored in directory: /root/.cache/pip/wheels/68/88/65/58d6aa058698c47d7a24352952ea07a1409cc2eff3a8087c0c
Successfully built scorecardpy


In [1]:
!pip install xverse

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from xverse.transformer import WOE
import scorecardpy as sc

In [4]:
data = pd.read_csv('cleaned_data.csv')

In [ ]:
data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Amount_log,Value_log
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0,6.908755,6.908755
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0,NaN,3.044522
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0,6.216606,6.216606
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0,9.903538,9.989711
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0,NaN,6.469250


In [ ]:
data.isnull().sum()

,0
TransactionId,0
BatchId,0
AccountId,0
SubscriptionId,0
CustomerId,0
ProviderId,0
ProductId,0
ProductCategory,0
ChannelId,0
Amount,0


In [5]:
data['Is_Positive_Amount'] = data['Amount'] > 0
data['Amount_log'] = np.where(data['Amount'] > 0, np.log1p(data['Amount']), 0)

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
data.isnull().sum()

,0
TransactionId,0
BatchId,0
AccountId,0
SubscriptionId,0
CustomerId,0
ProviderId,0
ProductId,0
ProductCategory,0
ChannelId,0
Amount,0


In [6]:
# Aggregate Features
agg_features = data.groupby('CustomerId')['Amount'].agg([
    ('Total_Transaction_Amount', 'sum'),
    ('Avg_Transaction_Amount', 'mean'),
    ('Transaction_Count', 'count'),
    ('Std_Transaction_Amount', 'std')
]).reset_index()

In [7]:
data = data.merge(agg_features, on='CustomerId', how='left')

In [8]:
# Convert TransactionStartTime to datetime
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])

# Extract features
data['Transaction_Hour'] = data['TransactionStartTime'].dt.hour
data['Transaction_Day'] = data['TransactionStartTime'].dt.day
data['Transaction_Month'] = data['TransactionStartTime'].dt.month
data['Transaction_Year'] = data['TransactionStartTime'].dt.year

In [9]:
# Label Encoding for ordinal encoding
label_encoder = LabelEncoder()
categorical_cols = ['ProviderId', 'ProductCategory', 'ChannelId']
for col in categorical_cols:
    data[col + '_Encoded'] = label_encoder.fit_transform(data[col])

# One-Hot Encoding
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [ ]:
data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProductId,Amount,Value,TransactionStartTime,PricingStrategy,...,ProviderId_ProviderId_5,ProviderId_ProviderId_6,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,ProductId_10,1000.0,1000,2018-11-15 02:18:49+00:00,2,...,False,True,True,False,False,False,False,False,True,False
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,ProductId_6,-20.0,20,2018-11-15 02:19:08+00:00,2,...,False,False,False,False,True,False,False,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,ProductId_1,500.0,500,2018-11-15 02:44:21+00:00,2,...,False,True,True,False,False,False,False,False,True,False
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,ProductId_21,20000.0,21800,2018-11-15 03:32:55+00:00,2,...,False,False,False,False,False,False,True,False,True,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,ProductId_6,-644.0,644,2018-11-15 03:34:21+00:00,2,...,False,False,False,False,True,False,False,True,False,False


In [ ]:
data.isnull().sum()

,0
TransactionId,0
BatchId,0
AccountId,0
SubscriptionId,0
CustomerId,0
ProductId,0
Amount,0
Value,0
TransactionStartTime,0
PricingStrategy,0


In [10]:
data['Std_Transaction_Amount'].fillna(data['Std_Transaction_Amount'].mean(), inplace=True)

In [11]:
scaler = StandardScaler()
numerical_cols = ['Total_Transaction_Amount', 'Avg_Transaction_Amount',
                  'Transaction_Count', 'Std_Transaction_Amount', 'Amount_log', 'Value_log']

# Standardize numerical columns
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

In [20]:
exclude_columns = ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'TransactionStartTime']
data_for_binning = data.drop(columns=exclude_columns)

In [21]:
# Perform WOE binning
bins = sc.woebin(data_for_binning, y='FraudResult')
print(bins)

[INFO] creating woe binning ...
Binning on 95662 rows and 32 columns in 00:00:24
{'Is_Positive_Amount':              variable         bin  count  count_distr   good  bad   badprob  \
0  Is_Positive_Amount  [-inf,1.0)  38189     0.399208  38184    5  0.000131   
1  Is_Positive_Amount   [1.0,inf)  57473     0.600792  57285  188  0.003271   

        woe    bin_iv  total_iv breaks  is_special_values  
0 -2.736867  1.023740  1.204976    1.0              False  
1  0.484515  0.181235  1.204976    inf              False  , 'ChannelId_ChannelId_5':                 variable         bin  count  count_distr   good  bad  \
0  ChannelId_ChannelId_5  [-inf,inf)  95662          1.0  95469  193   

    badprob  woe  bin_iv  total_iv breaks  is_special_values  
0  0.002018  0.0     0.0       0.0    inf              False  , 'ProductCategory_financial_services':                              variable         bin  count  count_distr   good  \
0  ProductCategory_financial_services  [-inf,1.0)  50257      

In [22]:
# Apply WOE transformation
data_woe = sc.woebin_ply(data_for_binning, bins)

[INFO] converting into woe values ...
Woe transformating on 95662 rows and 31 columns in 00:00:12


In [23]:
data_woe.head()

,FraudResult,Is_Positive_Amount_woe,ChannelId_ChannelId_5_woe,ProductCategory_financial_services_woe,Total_Transaction_Amount_woe,ChannelId_ChannelId_3_woe,Transaction_Hour_woe,ProductCategory_utility_bill_woe,ProductCategory_Encoded_woe,Value_log_woe,...,ProductCategory_data_bundles_woe,Transaction_Day_woe,ProviderId_ProviderId_5_woe,Transaction_Year_woe,Amount_woe,ProductCategory_tv_woe,PricingStrategy_woe,Std_Transaction_Amount_woe,Avg_Transaction_Amount_woe,Amount_log_woe
0,0,0.484515,0.0,-1.154666,-3.891521,0.472374,-0.231181,0.0,-1.559112,-4.209236,...,0.0,-0.592431,-0.238414,-0.245002,-3.932398,0.0,0.094399,-2.806436,-3.071208,-3.918154
1,0,-2.736867,0.0,0.565446,-3.891521,-2.162969,-0.231181,0.0,0.569263,-4.171778,...,0.0,-0.592431,-0.238414,-0.245002,-2.736867,0.0,0.094399,-2.806436,-3.071208,-2.736867
2,0,0.484515,0.0,-1.154666,-2.862343,0.472374,-0.231181,0.0,-1.559112,-4.171778,...,0.0,-0.592431,-0.238414,-0.245002,-3.932398,0.0,0.094399,-2.806436,-3.071208,-3.918154
3,0,0.484515,0.0,-1.154666,-3.891521,0.472374,-0.231181,0.0,0.569263,1.143627,...,0.0,-0.592431,-0.238414,-0.245002,1.060141,0.0,0.094399,-2.806436,-3.420700,1.049062
4,0,-2.736867,0.0,0.565446,-3.891521,-2.162969,-0.231181,0.0,0.569263,-4.171778,...,0.0,-0.592431,-0.238414,-0.245002,-2.736867,0.0,0.094399,-2.806436,-3.420700,-2.736867


In [28]:
# Extract IV scores from the bins object
iv_df = pd.DataFrame({
    'variable': bins.keys(),
    'total_iv': [b['total_iv'].values[0] for b in bins.values()]
})

# Sort IV scores in descending order
iv_sorted = iv_df.sort_values(by='total_iv', ascending=False)
print("IV Scores Sorted by Importance:")
print(iv_sorted)

IV Scores Sorted by Importance:
                              variable  total_iv
29              Avg_Transaction_Amount  5.116476
28              Std_Transaction_Amount  4.979270
17                           ProductId  3.806318
3             Total_Transaction_Amount  3.736760
8                            Value_log  3.596932
15                  ProviderId_Encoded  3.278587
25                              Amount  2.714451
30                          Amount_log  2.685168
10                               Value  2.641190
14             ProviderId_ProviderId_6  1.220905
19             ProviderId_ProviderId_4  1.204976
0                   Is_Positive_Amount  1.204976
12               ChannelId_ChannelId_2  1.152927
4                ChannelId_ChannelId_3  0.945887
9                    ChannelId_Encoded  0.901043
7              ProductCategory_Encoded  0.828103
16             ProductCategory_airtime  0.816921
20                   Transaction_Count  0.673549
2   ProductCategory_financial_service

In [31]:
# Identify low-IV features
low_iv_features = iv_sorted[iv_sorted['total_iv'] < 0.02]['variable']
print("Features with low IV scores:", low_iv_features.tolist())

# Add "_woe" suffix to low_iv_features to match data_woe column names
low_iv_features_woe = [f'{col}_woe' for col in low_iv_features]

# Drop low-IV features from the WOE-transformed dataset
data_woe.drop(columns=low_iv_features_woe, inplace=True)

Features with low IV scores: ['ProductCategory_utility_bill', 'ChannelId_ChannelId_5', 'ProductCategory_tv', 'ProviderId_ProviderId_3', 'ProductCategory_data_bundles']


In [32]:
data_woe.to_csv('final_dataset.csv', index=False)